In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
import math
import plotly.graph_objects as go
from plotting import CandlePlot
from tqdm import tqdm
from tabulate import tabulate
import pickle as pkl
pd.set_option("display.max_columns", None)

In [3]:
class Data:
    
    def __init__(self, path):
        self.df = {
            'raw': pd.read_pickle(path)
        }
        if 'time' in self.df['raw'].columns:
            self.df['raw']['time'] = [ x.replace(tzinfo=None) for x in self.df['raw']['time']]

    def __repr__(self) -> str:
        repr = str()
        for name, df in self.df.items():
            repr = repr + name + ':\n' + str(df.head(3)) + '\n'
        return repr

    def shorten(self, name: str, rows: int, direction: int, source: str='raw', cols: list=None):
        '''Create new dataframe with specified list of columns and number of rows
        direction: 1 if data should be selected from top and -1 if from bottom
        '''
        assert (direction != 1 or direction != -1), 'direction must be 1 (top) or -1 (bottom)'
        
        if cols == None:
            cols = self.df[source].columns
        if direction == 1:
            self.df[name] = self.df[source][cols].iloc[:rows].copy()
        else:
            self.df[name] = self.df[source][cols].iloc[-rows:].copy()
        self.df[name].reset_index(drop=True, inplace=True)

    def add_columns(self, name: str, cols: list):
        '''Add new columns to component dataframes
        '''        
        exist_cols = list(self.df[name].columns)
        cols = exist_cols + cols
        self.df[name] = self.df[name].reindex(columns = cols) 

    def prepare_fast_data(self, name: str):
        '''Prepare data as an array for fast processing
        fcols = {col1: col1_index, col2: col2_index, .... }     
        fdata = [array[col1], array[col2], array[col3], .... ]
        Accessed by: self.fdata[fcols[column_name]] for whole column or
                     self.fdata[fcols[column_name]][row_index] for a specific row item
        '''
        self.fcols = dict()
        for i in range(len(self.df[name].columns)):
            self.fcols[self.df[name].columns[i]] = i
        self.fastdf = [self.df[name][col].array for col in self.df[name].columns]

    def fdata(self, column: str, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            return self.fastdf[self.fcols[column]]
        else:
            return self.fastdf[self.fcols[column]][index]
        
    def update_fdata(self, column: str, value, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            self.fastdf[self.fcols[column]] = value
        else:
            self.fastdf[self.fcols[column]][index] = value


In [4]:
PATH = 'D:/Trading/forex_bot/outputs/'

In [5]:
d = Data(PATH + "inputs.all.pkl")

In [6]:
d.df['raw']

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit
0,sim_1,1,1.2,0.001,1.5,True,2
1,sim_2,1,1.2,0.001,1.5,True,3
2,sim_3,1,1.2,0.001,1.5,True,4
3,sim_4,1,1.2,0.001,1.5,False,0
4,sim_5,1,1.2,0.002,1.5,True,2
...,...,...,...,...,...,...,...
859,sim_2284,-1,2.0,0.003,3.0,False,0
860,sim_2285,-1,2.0,0.004,3.0,True,8
861,sim_2286,-1,2.0,0.004,3.0,True,9
862,sim_2287,-1,2.0,0.004,3.0,True,10


In [7]:
cols = ['final_ac_bal', 'total_trades', 'avg_trade', 'avg_trade_duration', 'win_%', 'loss_%', 'expectancy', 
        'max_ac_bal', 'min_ac_bal', 'max_margin_used', 'total_streaks', 'avg_trades_per_streak',
        'total_wins', 'avg_win', 'avg_win_duration', 'total_losses', 'avg_loss', 'avg_loss_duration',
        'total_long', 'avg_long', 'avg_long_duration',
        'total_long_wins', 'avg_long_win', 'avg_long_win_duration',
        'total_long_losses', 'avg_long_loss', 'avg_long_loss_duration',
        'total_short', 'avg_short', 'avg_short_duration',
        'total_short_wins', 'avg_short_win', 'avg_short_win_duration',
        'total_short_losses', 'avg_short_loss', 'avg_short_loss_duration',
        
        ]
d.add_columns('raw', cols)
d.df['raw'].head(3)

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,avg_win,avg_win_duration,total_losses,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_1,1,1.2,0.001,1.5,True,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sim_2,1,1.2,0.001,1.5,True,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sim_3,1,1.2,0.001,1.5,True,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
d.prepare_fast_data('raw')

In [9]:
d.fdata('sim_name', 0)

'sim_1'

In [10]:
temp = pd.read_pickle(PATH + d.fdata('sim_name', 0) + '.pkl')
temp

{'sim_name': 'sim_1',
 'init_signal': 1,
 'cushion': 1.2,
 'risk': 0.001,
 'rr': 1.5,
 'margin_closeout': True,
 'streak_limit': 2,
 'results':                       time    bid_o    bid_h    bid_l    bid_c    ask_o  \
 0      2016-01-07 00:00:00  1.07757  1.07802  1.07750  1.07777  1.07772   
 1      2016-01-07 00:05:00  1.07779  1.07811  1.07755  1.07802  1.07798   
 2      2016-01-07 00:10:00  1.07803  1.07823  1.07803  1.07819  1.07822   
 3      2016-01-07 00:15:00  1.07815  1.07821  1.07790  1.07790  1.07834   
 4      2016-01-07 00:20:00  1.07787  1.07789  1.07768  1.07782  1.07804   
 ...                    ...      ...      ...      ...      ...      ...   
 594298 2023-12-29 21:35:00  1.10355  1.10374  1.10355  1.10371  1.10374   
 594299 2023-12-29 21:40:00  1.10370  1.10379  1.10356  1.10371  1.10388   
 594300 2023-12-29 21:45:00  1.10371  1.10384  1.10371  1.10378  1.10388   
 594301 2023-12-29 21:50:00  1.10379  1.10389  1.10375  1.10383  1.10393   
 594302 2023-12-29 21

In [11]:
temp['results'].tail(1)

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_c,signal,streak_no,trade_no,position_size,cum_position_size,buy_price,sell_price,buy_tp_price,sell_tp_price,unrealised_pl,realised_pl,ac_bal,margin_used,margin_closeout
594302,2023-12-29 21:55:00,1.10382,1.10388,1.10346,1.10363,1.10401,1.10406,1.10381,1.10386,1.10374,0.0,21950.0,1.0,100.0,100.0,1.10558,1.10458,1.10708,1.10308,0.084,0.0,363.4008,3.333333,363.4848


In [12]:
def populate_results(d: Data, i: int, df: pd.DataFrame):
    d.update_fdata('final_ac_bal', df['ac_bal'].iloc[-1], i)
    d.update_fdata('max_ac_bal', df['ac_bal'].max(), i)
    d.update_fdata('min_ac_bal', df['ac_bal'].min(), i)
    d.update_fdata('max_margin_used', df['margin_used'].max(), i)
    d.update_fdata('total_streaks', df['streak_no'].iloc[-1], i)
    d.update_fdata('avg_trades_per_streak', df.groupby('streak_no')['trade_no'].max().mean(), i)
    
    trades = df[df.signal != 0].copy()
    trades['return'] = trades['ac_bal'].shift(-1) - trades['ac_bal']
    trades['index'] = trades.index
    trades['duration'] = trades['index'].shift(-1) - trades['index']
    trades.dropna(inplace=True)

    d.update_fdata('total_trades', trades.shape[0], i)
    d.update_fdata('avg_trade', trades['return'].mean(), i)
    d.update_fdata('avg_trade_duration', trades['duration'].mean(), i)
    
    wins = trades[trades['return'] > 0]
    d.update_fdata('total_wins', wins.shape[0], i)
    d.update_fdata('avg_win', wins['return'].mean(), i)
    d.update_fdata('avg_win_duration', wins['duration'].mean(), i)
    d.update_fdata('win_%', d.fdata('total_wins', i) / d.fdata('total_trades', i), i)

    losses = trades[trades['return'] < 0]
    d.update_fdata('total_losses', losses.shape[0], i)
    d.update_fdata('avg_loss', losses['return'].mean(), i)
    d.update_fdata('avg_loss_duration', losses['duration'].mean(), i)
    d.update_fdata('loss_%', d.fdata('total_losses', i) / d.fdata('total_trades', i), i)
    
    longs = trades[trades.signal == 1]
    d.update_fdata('total_long', longs.shape[0], i)
    d.update_fdata('avg_long', longs['return'].mean(), i)
    d.update_fdata('avg_long_duration', longs['duration'].mean(), i)

    long_wins = longs[longs['return'] > 0]
    d.update_fdata('total_long_wins', long_wins.shape[0], i)
    d.update_fdata('avg_long_win', long_wins['return'].mean(), i)
    d.update_fdata('avg_long_win_duration', long_wins['duration'].mean(), i)

    long_losses = longs[longs['return'] < 0]
    d.update_fdata('total_long_losses', long_losses.shape[0], i)
    d.update_fdata('avg_long_loss', long_losses['return'].mean(), i)
    d.update_fdata('avg_long_loss_duration', long_losses['duration'].mean(), i)

    shorts = trades[trades.signal == -1]
    d.update_fdata('total_short', shorts.shape[0], i)
    d.update_fdata('avg_short', shorts['return'].mean(), i)
    d.update_fdata('avg_short_duration', shorts['duration'].mean(), i)

    short_wins = shorts[shorts['return'] > 0]
    d.update_fdata('total_short_wins', short_wins.shape[0], i)
    d.update_fdata('avg_short_win', short_wins['return'].mean(), i)
    d.update_fdata('avg_short_win_duration', short_wins['duration'].mean(), i)

    short_losses = shorts[shorts['return'] < 0]
    d.update_fdata('total_short_losses', short_losses.shape[0], i)
    d.update_fdata('avg_short_loss', short_losses['return'].mean(), i)
    d.update_fdata('avg_short_loss_duration', short_losses['duration'].mean(), i)


    d.update_fdata('expectancy', d.fdata('win_%', i) * d.fdata('avg_win', i) + d.fdata('loss_%', i) * d.fdata('avg_loss', i), i)

In [13]:
sims = d.df['raw'].shape[0]
for i in tqdm(range(sims), desc=" Analysing... "):
# for i in tqdm(range(2), desc=" Analysing... "):
    data = pd.read_pickle(PATH + d.fdata('sim_name', i) + '.pkl')
    assert (d.fdata('sim_name', i) == data['sim_name'] and
            d.fdata('init_signal', i) == data['init_signal'] and
            d.fdata('cushion', i) == data['cushion'] and
            d.fdata('risk', i) == data['risk'] and
            d.fdata('rr', i) == data['rr'] and
            d.fdata('margin_closeout', i) == data['margin_closeout'] and
            d.fdata('streak_limit', i) == data['streak_limit']), f"Parameters mismatch for {d.fdata('sim_name', i)}"
    
    populate_results(d, i, data['results'])

 Analysing... : 100%|██████████| 864/864 [02:00<00:00,  7.17it/s]


In [14]:
with open(PATH + 'hedging_sim_analysis.pkl', 'wb') as file:
        pkl.dump(d.df['raw'], file)

In [15]:
d.df['raw'].head()

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,avg_win,avg_win_duration,total_losses,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_1,1,1.2,0.001,1.5,True,2,363.40080,37686.0,-0.016892,15.768959,0.322879,0.675423,-0.016892,1001.04500,3.618676e+02,3.333333e+00,21950.0,1.716948,12168.0,0.161778,22.242850,25454.0,-0.102346,12.647835,18811.0,-0.017223,15.984477,6018.0,0.163047,22.966434,12765.0,-0.102248,12.671210,18875.0,-0.016563,15.554172,6150.0,0.160535,21.534797,12689.0,-0.102444,12.624320
1,sim_2,1,1.2,0.001,1.5,True,3,419.85380,36512.0,-0.015889,16.276649,0.327564,0.671450,-0.015889,1002.43652,4.172039e+02,4.800000e+00,16818.0,2.171067,11960.0,0.189565,22.353763,24516.0,-0.116142,13.297112,18123.0,-0.016423,16.273134,5888.0,0.191089,22.690387,12220.0,-0.116430,13.170049,18389.0,-0.015363,16.280113,6072.0,0.188088,22.027339,12296.0,-0.115857,13.423390
2,sim_3,1,1.2,0.001,1.5,True,4,305.59328,36166.0,-0.019201,16.432340,0.330255,0.669496,-0.019201,1002.25748,3.049337e+02,8.666667e+00,14738.0,2.453996,11944.0,0.227217,22.573175,24213.0,-0.140763,13.399744,18032.0,-0.019116,16.502939,5936.0,0.229282,23.221698,12094.0,-0.141038,13.206218,18134.0,-0.019285,16.362137,6008.0,0.225177,21.932423,12119.0,-0.140488,13.592871
3,sim_4,1,1.2,0.001,1.5,False,0,-856608.05235,35475.0,-24.174998,16.752417,0.333531,0.666469,-24.174998,2998.61543,-2.750048e+06,3.756416e+07,11833.0,2.998056,11832.0,169.608752,22.190754,23643.0,-121.152933,14.030834,17695.0,59.387389,16.812150,5873.0,336.127133,22.841989,11822.0,-78.092946,13.816613,17780.0,-107.337902,16.692970,5959.0,5.493555,21.548918,11821.0,-164.216563,14.245072
4,sim_5,1,1.2,0.002,1.5,True,2,877.79760,10101.0,-0.012098,58.832888,0.337194,0.661123,-0.012098,1001.72920,8.718790e+02,3.333333e+00,5996.0,1.684790,3406.0,0.306745,79.558426,6678.0,-0.174749,48.233004,5039.0,-0.010783,58.134749,1690.0,0.309888,78.738462,3341.0,-0.173016,47.685124,5062.0,-0.013407,59.527855,1716.0,0.303648,80.365967,3337.0,-0.176484,48.781540


In [16]:
df = d.df['raw']

In [17]:
df.sort_values(by='expectancy', ascending=False)

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,avg_win,avg_win_duration,total_losses,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
531,sim_1244,-1,2.0,0.001,1.5,False,0,4.793868e+08,35474.0,13513.722106,16.752889,0.333540,0.666460,13513.722106,4.793868e+08,-4.038322e+09,8.963510e+10,11833.0,2.997972,11832.0,383726.828212,22.189571,23642.0,-171765.081356,14.032019,17694.0,199386.367675,16.812592,5873.0,772479.806426,22.839775,11821.0,-85342.315667,13.818120,17780.0,-171459.876922,16.693476,5959.0,584.314191,21.548750,11821.0,-258187.847046,14.245918
243,sim_244,-1,2.0,0.001,1.5,False,0,4.793868e+08,35474.0,13513.722106,16.752889,0.333540,0.666460,13513.722106,4.793868e+08,-4.038322e+09,8.963510e+10,11833.0,2.997972,11832.0,383726.828212,22.189571,23642.0,-171765.081356,14.032019,17694.0,199386.367675,16.812592,5873.0,772479.806426,22.839775,11821.0,-85342.315667,13.818120,17780.0,-171459.876922,16.693476,5959.0,584.314191,21.548750,11821.0,-258187.847046,14.245918
819,sim_2244,-1,2.0,0.001,1.5,False,0,4.793868e+08,35474.0,13513.722106,16.752889,0.333540,0.666460,13513.722106,4.793868e+08,-4.038322e+09,8.963510e+10,11833.0,2.997972,11832.0,383726.828212,22.189571,23642.0,-171765.081356,14.032019,17694.0,199386.367675,16.812592,5873.0,772479.806426,22.839775,11821.0,-85342.315667,13.818120,17780.0,-171459.876922,16.693476,5959.0,584.314191,21.548750,11821.0,-258187.847046,14.245918
675,sim_2100,1,2.0,0.001,1.5,False,0,4.793868e+08,35475.0,13513.341172,16.752417,0.333531,0.666469,13513.341172,4.793868e+08,-4.038322e+09,8.963510e+10,11833.0,2.998056,11832.0,383726.828240,22.190754,23643.0,-171757.816422,14.030834,17695.0,199375.099736,16.812150,5873.0,772479.806479,22.841989,11822.0,-85335.096737,13.816613,17780.0,-171459.876927,16.692970,5959.0,584.314194,21.548918,11821.0,-258187.847056,14.245072
387,sim_1100,1,2.0,0.001,1.5,False,0,4.793868e+08,35475.0,13513.341172,16.752417,0.333531,0.666469,13513.341172,4.793868e+08,-4.038322e+09,8.963510e+10,11833.0,2.998056,11832.0,383726.828240,22.190754,23643.0,-171757.816422,14.030834,17695.0,199375.099736,16.812150,5873.0,772479.806479,22.841989,11822.0,-85335.096737,13.816613,17780.0,-171459.876927,16.692970,5959.0,584.314194,21.548918,11821.0,-258187.847056,14.245072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,sim_52,1,1.5,0.001,1.5,False,0,-7.683140e+06,35475.0,-216.607180,16.752417,0.333531,0.666469,-216.607180,1.631968e+05,-5.278497e+07,8.947849e+08,11833.0,2.998056,11832.0,3887.497683,22.190754,23643.0,-2270.482270,14.030834,17695.0,1712.316586,16.812150,5873.0,7797.541232,22.841989,11822.0,-1310.735718,13.816613,17780.0,-2136.309433,16.692970,5959.0,33.883693,21.548918,11821.0,-3230.310012,14.245072
627,sim_2052,1,1.5,0.001,1.5,False,0,-7.683140e+06,35475.0,-216.607180,16.752417,0.333531,0.666469,-216.607180,1.631968e+05,-5.278497e+07,8.947849e+08,11833.0,2.998056,11832.0,3887.497683,22.190754,23643.0,-2270.482270,14.030834,17695.0,1712.316586,16.812150,5873.0,7797.541232,22.841989,11822.0,-1310.735718,13.816613,17780.0,-2136.309433,16.692970,5959.0,33.883693,21.548918,11821.0,-3230.310012,14.245072
195,sim_196,-1,1.5,0.001,1.5,False,0,-7.683140e+06,35474.0,-216.613287,16.752889,0.333540,0.666460,-216.613287,1.631968e+05,-5.278497e+07,8.947849e+08,11833.0,2.997972,11832.0,3887.497667,22.189571,23642.0,-2270.578298,14.032019,17694.0,1712.413357,16.812592,5873.0,7797.541200,22.839775,11821.0,-1310